# Introduction to Jupyter Notebooks

Jupyter Notebooks have become a mainstream tool for interactive computing in every field of science. Jupyter is slowly making its way into HPC, and PDC has recently started offering access to Jupyter on the Tegner cluster.

In this lesson, we will cover: 
- Necessary configuration steps needed to run Jupyter on Tegner@PDC
- How to start a Jupyter server on a compute node
- Basics of Jupyter: What is it? How can we use it?

In following lessons, we will see:
- How to launch jobs via Jupyter to the Slurm scheduler
- How to monitor and analyze simulation results generated by other software on the fly in Jupyter
- Some parallel computing concepts in Python using `multiprocessing`, `ipyparallel` and `mpi4py`

## First fetch the sources

Log in to Tegner, create a new directory and clone the repository with the lesson material:

```bash
$ ssh <username>@tegner.pdc.kth.se
$ cd /cfs/klemming/nobackup/<initial>/<username>
$ mkdir prace-jupyter
$ cd prace-jupyter
$ git clone https://github.com/PDC-support/jupyter-notebook.git
```

## Setting up Jupyter on PDC

To connect from your browser to a Jupyter Notebook server running on a Tegner compute node you need to follow a few steps.
First make sure that the connection between your browser and the Jupyter server running at PDC is secure, by:
- Generating a Jupyter configuration file
- Setting a strong Jupyter password
- Setting up a self-signed SSL certificate to enable https

### Jupyter configuration

+ Load module ``anaconda/py36/5.0.1`` and activate the ``prace`` environment.

```bash
$ module load anaconda/py36/5.0.1
$ source activate prace
```

+ Generate the configuration file `$HOME/.jupyter`:

```bash
$ jupyter notebook --generate-config
```

### Setting a Jupyter password
+ Create your Jupyter password 

```
$ jupyter notebook password
```

+ The entered password will be saved in hashed form in the file `$HOME/.jupyter/jupyter_notebook_config.json`.

### Generate a SSL certificate

+ Generate a self-signed certificate with `openssl` in your `Private` directory: 

```bash
$ cd $HOME/Private
$ openssl req -x509 -nodes -days 30 -newkey rsa:2048 -keyout mykey.key -out mycert.pem
```

+ Add information on the certificate in your Jupyter config file:

```
$ c.NotebookApp.certfile = u'/afs/pdc.kth.se/home/<initial>/<username>/Private/mycert.pem'
$ c.NotebookApp.keyfile = u'/afs/pdc.kth.se/home/<initial>/<username>/Private/mykey.key'
```

### Interactive use

+ Start Jupyter notebook with port, host IP, and secure protocol

```bash
$ ipnport=$(shuf -i8000-9999 -n1)
$ ipnip=$(hostname -i)
$ jupyter-notebook --no-browser --port=$ipnport --ip=$ipnip --certfile=mycert.pem --keyfile mykey.key
```

+ Jupyter notebook will print output like:

```
The Jupyter Notebook is running at:
https://<IP.address>:<port>/
```

+ Copy the **https** link to your browser. Your browser will likely complain that your self-signed certificate is not secure or not recognized. Tell it to proceed anyways.


+ The notebook is ready for use after entering your password!

### Batch job

- Copy/paste this batch script to a file `submit_jupyter.bash`:

```bash
#!/bin/bash
#SBATCH -A edu18.prace
#SBATCH --nodes 1
#SBATCH --time 1:00:00
#SBATCH --job-name tunnel
#SBATCH --output jupyter-log-%J.txt

module load anaconda/py36/5.0.1
source activate prace

XDG_RUNTIME_DIR=""
ipnport=$(shuf -i8000-9999 -n1)
ipnip=$(hostname -i)

echo -e "
    Copy/Paste this in your local terminal to ssh tunnel with remote
    -----------------------------------------------------------------
    ssh -N -L $ipnport:$ipnip:$ipnport $USER@tegner.pdc.kth.se
    -----------------------------------------------------------------

    Then open a browser on your local machine to the following address
    ------------------------------------------------------------------
    https://localhost:$ipnport  
    ------------------------------------------------------------------
    "

jupyter-notebook --no-browser --port=$ipnport --ip=$ipnip
```

- Submit the batch script by 

```bash
$ sbatch submit_jupyter.bash
```

- Check the instructions in `jupyter-log-%J.txt`:
    - Create the ssh tunnel from a new terminal from your computer: `ssh -N -L <port>:<ip>:<port> <username>@tegner.pdc.kth.se`.
    - Copy/paste the URL `https://localhost:<port>` provided in the output file into your browser. 

## Introduction

+ What is Jupyter notebook?

+ Why do we want to use Jupyter notebook?

+ Features?